Image Processing

In [1]:
import glob
import PIL.Image
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import matplotlib.image as mpimg
import matplotlib.patches as patches
from IPython.core.debugger import Tracer
import cv2

In [2]:
class Constants:
    # The paths below recently changed.. so be sure to consider that...
    orig_images_root = "/home/apil/data/hw_forms/initial_data/s3_upload1/"
    labels_path = "/home/apil/data/hw_forms/initial_data/labels.csv"
    labels_renamed_path = "/home/apil/data/hw_forms/initial_data/labels_renamed.csv"
    masks_root = "/home/apil/data/hw_forms/train/masks/"
    greyscale_root = "/home/apil/data/hw_forms/train/images/"

In [3]:
'''
Run the following so that all files are changed to have .jpg format:
----------------------------
for f in *.jpeg; do 
mv -- "$f" "${f%.jpeg}.jpg"
done
----------------------------
for f in *.png; do 
mv -- "$f" "${f%.png}.jpg"
done
----------------------------
for f in *.JPG; do 
mv -- "$f" "${f%.JPG}.jpg"
done
'''
pass

In [5]:
'''
Run the following so that file references in labels.csv are changed 
so that all images are referenced with their .jpg extensions
'''

"""
def modifyLabels():
    counter = 0;
    g = open(Constants.labels_renamed_path, "w")
    with open(Constants.labels_path) as f:
        for line in f:
            line = line.replace(".jpeg", ".jpg")
            line = line.replace(".png", ".jpg")
            line = line.replace(".JPG",".jpg")
            counter +=1
            g.write(line)
    print("wrote {:d} lines.".format(counter))
"""

'''
Run the below to modify the labels, so that all images
with .jpeg, .png, .JPG files are renamed aptly to .jpg
----------------------------
modifyLabels()
----------------------------
'''
pass

In [6]:
class FormImage:

    def __init__(self, name, boxes):
        self.name = name
        self.boxes = boxes 

In [7]:
class Box:  
    
    def __init__(self, tL, tR, bL, bR):
        
        self.x0 = tL
        self.y0 = tR
        
        self.x1 = bL
        self.y1 = tR
        
        self.x2 = bL
        self.y2 = bR
        
        self.x3 = tL
        self.y3 = bR
    
    def toStr(self):
        return " tL: ({:d},{:d}), tR: ({:d},{:d}), bR: ({:d},{:d}), bL: ({:d},{:d}) ".format( \
                                            self.x0, self.y0, self.x1, self.y1, self.x2, self.y2, self.x3, self.y3)

    def evalPatchParams(self, orig_ht, resize_ht, orig_wd, resize_wd):
        box_ht = self.y2 - self.y1
        box_wd = self.x2 - self.x0
        
        print("box corners: {}".format(self.toStr()))
        print("box ht: {}, box wd: {}".format(box_ht, box_wd))
        print("orig_ht: {}, orig_wd: {}, resize_ht: {}, resize_wd: {}".format(orig_ht, orig_wd, resize_ht, resize_wd))
        
        ht_rescale = resize_ht/orig_ht 
        wd_rescale = resize_wd/orig_wd 

        new_x0 = self.x0 * wd_rescale
        new_y0 = self.y0 * ht_rescale

        new_ht = box_ht * ht_rescale
        new_wd = box_wd * wd_rescale
        
        return (new_x0, new_y0), new_wd, new_ht

    @classmethod
    def fromStr(cls, string, imgName):
        temp = [val.strip() for val in string.split(',')]
        try:
            return cls(Box.num(temp[0]), Box.num(temp[1]), Box.num(temp[2]), Box.num(temp[3]))
        except ValueError as e:            
            # print("failed to parse to int. Passed string is: {}, ImageName is: {}".format(string, imgName))
            pass
            
    
    @staticmethod
    def num(s):
        try:
            return int(s)
        except:
            # print("ERROR trying to parse int from string: {}".format(s))
            raise ValueError("ERROR trying to parse int from string: {}".format(s))

In [8]:
class Utils:
    
    @staticmethod
    def extractName(line):
        imgName = line[line.rfind('/')+1 : line.rfind('.jpg')]
        return imgName
    
    @staticmethod
    def extractCoords(line):
        coords = line[line.rfind('.jpg,')+5 : line.rfind(',qr')]
        return coords  

In [10]:

class AllImages:
    
    def __init__(self):
        self.initFromLabels();
    
    def add(self, formImage):
        formImages.append(formImage)
    
    def initFromLabels(self):
        self.imageToBoxes = {}
        with open(Constants.labels_renamed_path) as f:
            for line in f:
                imgName = Utils.extractName(line)
                coords = Utils.extractCoords(line)
                
                box = Box.fromStr(coords, imgName)                                       
                imgBoxes = self.imageToBoxes.get(imgName, [])
                imgBoxes.append(box)
                self.imageToBoxes[imgName] = imgBoxes
                
    def getBoxes(self, imgName):
        return self.imageToBoxes.get(imgName,[])

In [11]:
def createMask(cv2Img, imgName, boxes):
    ht, wd = cv2Img.shape
    arr = np.zeros((ht, wd), dtype=np.uint8)
    for box in boxes:
        arr[box.y0: box.y3+1, box.x0: box.x1+1] = 255

    destImg = Constants.masks_root + imgName + ".jpg"        
    im2 = cv2.imwrite(destImg, arr)

In [12]:
def displayImage(imgPath):
    img=mpimg.imread(imgPath)
    imgplot = plt.imshow(img)
    plt.show()    

In [13]:
class ImageProcessor:

    def __init__(self):
        self.allImages = AllImages()
        
    def processImages(self):
        cntGood = 0
        cntBad = 0
        
        for infile in glob.glob(Constants.orig_images_root + "/*.jpg"):

            try:
                imgName = Utils.extractName(infile)
                boxes = self.allImages.getBoxes(imgName)

                # do not process image if no qr codes found.
                if(boxes is None or len(boxes) == 0):
                    cntBad +=1
                    print("skipping image: {} because ZERO boxes were found".format(imgName))
                    print("----------------------------------") 
                    continue

                outfile = Constants.greyscale_root +imgName+".jpg"


                print("processing file: {}".format(infile))   
                print("boxes found: {}".format(len(boxes)))
                im = cv2.imread(infile, cv2.IMREAD_GRAYSCALE)

                # Create a mask of the image... and save it. The height and width of
                # the mask would be akin to the original. Use the given boxes coordinates
                # to create the bounding rectangles. Then, save it as "imgName.mask.png"
                createMask(im, imgName, boxes)
                cv2.imwrite(outfile, im)

                print("DONE! ")
                cntGood +=1
            except:
                cntBad +=1
                print("cannot create thumbnail for {}".format(infile))

            print("----------------------------------") 
            
        print("======> SUMMARY: Processed successfully: {:d} files and unsuccessfully: {:d} files.".format(cntGood, cntBad))


    def displayResizedImagesAndBoxes(self, imgName):

        boxes = self.allImages.getBoxes(imgName)
        resized_img = Constants.resized_images_root + imgName + ".jpg"
        orig_img =  Constants.orig_images_root + imgName + ".jpg"

        orig_ht, orig_wd, orig_depth = plt.imread(orig_img).shape

        dpi = 80
        im_data = plt.imread(resized_img)
        height, width, depth = im_data.shape

        # What size does the figure need to be in inches to fit the image?
        figsize = width / float(dpi), height / float(dpi)

        # Create a figure of the right size with one axes that takes up the full figure
        fig = plt.figure(figsize=figsize)
        ax = fig.add_axes([0, 0, 1, 1])

        # create bounding boxes for the QR codes
        print("=======> BOXES FOUND: {:d}".format(len(boxes)));
        for box in boxes:
            xy,wd,ht = box.evalPatchParams(orig_ht, height, orig_wd, width)
            rect = patches.Rectangle(xy,wd,ht,linewidth=1,edgecolor='r',facecolor='none')
            print("adding rectangle at: {}, with width: {} and ht: {}".format(xy[0], xy[1], wd, ht))
            ax.add_patch(rect)

        # Hide spines, ticks, etc.
        ax.axis('off')

        # Display the image.
        ax.imshow(im_data, cmap='gray')

        plt.show()                

In [14]:
processor = ImageProcessor()

In [15]:
processor.processImages()

processing file: /home/apil/data/hw_forms/initial_data/s3_upload1/3ZQA3IO31AQ7WQ1D1NM28SCN43U1O6-3L2IS5HSFAI8AMQTSY1BSPC3D78UNB-A22ZV1G6YEIK89-hit.jpg
boxes found: 6
DONE! 
----------------------------------
processing file: /home/apil/data/hw_forms/initial_data/s3_upload1/3ZQA3IO31AQ7WQ1D1NM28SCN43U1O6-3TEM0PF1Q5XJ8ON05X9O7DC2JE1D08-AKP3UXJ0TDPVO-ABCscan.jpg
boxes found: 6
DONE! 
----------------------------------
processing file: /home/apil/data/hw_forms/initial_data/s3_upload1/3ZQA3IO31AQ7WQ1D1NM28SCN43U1O6-3C6FJU71TQT2MLU7Z0AM95AAHSPUYG-A1CNITFHQHCTZ1-53774228473__B13613D7-DBC1-4670-95BB-4B3091A7AB2A.jpg
boxes found: 6
DONE! 
----------------------------------
processing file: /home/apil/data/hw_forms/initial_data/s3_upload1/3ZQA3IO31AQ7WQ1D1NM28SCN43U1O6-3AMYWKA6YBM5QW04XV3LR8YS0W96OG-A3D4YRQRF43YPL-MTURK_0115181449_HDR.jpg
boxes found: 6
DONE! 
----------------------------------
processing file: /home/apil/data/hw_forms/initial_data/s3_upload1/3ZQA3IO31AQ7WQ1D1NM28SCN43U1O6-3LO69